<a href="https://colab.research.google.com/github/Jerry086/SALSA/blob/jerry/Colab/AudioSet_label_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [18]:
audio_metadata = pd.read_csv('/content/training_metadata.csv')
audio_metadata['labels'] = audio_metadata['labels'].apply(eval)
audio_metadata

,video_id,start_time_seconds,end_time_seconds,labels
0,wqoOX8K8DEU,30.0,40.0,"[396, 397]"
1,wqH6Sj_h948,120.0,130.0,"[0, 441, 443]"
2,wq1098my4zA,130.0,140.0,"[27, 137, 271]"
3,wqR7LHho-WE,10.0,20.0,"[0, 22, 25]"
4,wq6Me-UUbSc,360.0,370.0,[413]
...,...,...,...,...
21777,2w6tV5kDGWo,240.0,250.0,[178]
21778,2wZCoeq9Ppc,80.0,90.0,"[137, 138, 185, 195, 196, 198, 268]"
21779,2wajg-UP-Gs,0.0,10.0,[459]
21780,lZavPVn7O4Q,180.0,190.0,"[137, 258, 260, 273]"


In [4]:
!git clone https://github.com/audioset/ontology.git

Cloning into 'ontology'...
remote: Enumerating objects: 14, done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 14
Receiving objects: 100% (14/14), 87.20 KiB | 1.64 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [5]:
label_index = pd.read_csv('/content/class_labels_indices.csv')
label_index

,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/05zppz,"Male speech, man speaking"
2,2,/m/02zsn,"Female speech, woman speaking"
3,3,/m/0ytgt,"Child speech, kid speaking"
4,4,/m/01h8n0,Conversation
...,...,...,...
522,522,/m/07p_0gm,Throbbing
523,523,/m/01jwx6,Vibration
524,524,/m/07c52,Television
525,525,/m/06bz3,Radio


In [7]:
# Build a dictionary to map mid to index
mid_to_index = {row['mid']: row['index'] for _, row in label_index.iterrows()}
# mid_to_index

In [ ]:
import json

# Load the ontology JSON data
ontology_path = "/content/ontology/ontology.json"
with open(ontology_path, 'r') as file:
    ontology_data = json.load(file)

# Build a dictionary to map child ID to parent ID
child_to_parent = {}
for item in ontology_data:
    for child_id in item["child_ids"]:
      child_index = mid_to_index.get(child_id, None)
      parent_index = mid_to_index.get(item["id"], None)
      if not parent_index or not child_index:
        continue
      child_to_parent[child_index] = parent_index

child_to_parent

In [ ]:
node_to_root = {}
# Function to find root label
def find_root(node):
  while node in child_to_parent:
    node = child_to_parent[node]
  return node

for k in child_to_parent:
  node_to_root[k] = find_root(k)

node_to_root

In [22]:
for i, row in audio_metadata.iterrows():
  labels = row['labels']
  new_labels = set()
  for label in labels:
    new_labels.add(node_to_root.get(label, label))
  audio_metadata.at[i, 'labels'] = list(new_labels)

In [23]:
audio_metadata

,video_id,start_time_seconds,end_time_seconds,labels
0,wqoOX8K8DEU,30.0,40.0,[388]
1,wqH6Sj_h948,120.0,130.0,"[0, 441]"
2,wq1098my4zA,130.0,140.0,"[137, 27, 271]"
3,wqR7LHho-WE,10.0,20.0,"[0, 25, 22]"
4,wq6Me-UUbSc,360.0,370.0,[404]
...,...,...,...,...
21777,2w6tV5kDGWo,240.0,250.0,[137]
21778,2wZCoeq9Ppc,80.0,90.0,"[137, 268]"
21779,2wajg-UP-Gs,0.0,10.0,[458]
21780,lZavPVn7O4Q,180.0,190.0,"[137, 258, 260, 273]"


In [24]:
from google.colab import files
# save updated metadata
audio_metadata.to_csv('/content/training_metadata_rootclass.csv', index=False)
files.download('/content/training_metadata_rootclass.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>